In [ ]:
import os
import pandas as pd
import torch
import dgl




df = pd.read_csv(os.path.join("tables","graph.csv"))

# 1) Mapear equipos a IDs de nodo
equipos = pd.unique(df[["equipo_in", "equipo_out"]].values.ravel())
equipo2id = {e: i for i, e in enumerate(equipos)}
num_nodes = len(equipos)

# 2) Mapear (comp, stat) a IDs de relación
df["rel_name"] = df["stat"]
rel_names = df["rel_name"].unique()
rel2id = {r: i for i, r in enumerate(rel_names)}
num_rels = len(rel2id)

# 3) Construir listas src, dst y etype
src = [equipo2id[e] for e in df["equipo_in"]]
dst = [equipo2id[e] for e in df["equipo_out"]]
etype = [rel2id[r] for r in df["rel_name"]]

src = torch.tensor(src, dtype=torch.int64)
dst = torch.tensor(dst, dtype=torch.int64)
etype = torch.tensor(etype, dtype=torch.int64)

# 4) Crear grafo homogéneo
g = dgl.graph((src, dst), num_nodes=num_nodes)

# Guardar el ID de tipo de relación en la arista (lo que tu RGCN usa)
g.edata[dgl.ETYPE] = etype

# 5) Factor de normalización por nodo destino
g.edata["norm"] = dgl.norm_by_dst(g).unsqueeze(1)  # (E, 1)

print(g)
print("num_nodes:", g.num_nodes())
print("num_edges:", g.num_edges())
print("num_rels:", num_rels)